In [2]:
import os
from telegram.ext  import Updater, CommandHandler, MessageHandler, Filters
import string
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
import annoy
from gensim.models import Word2Vec, FastText
import pickle
import numpy as np
from tqdm.notebook import tqdm
import pandas as pd
import pickle
    
tqdm.pandas()

from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)

C:\Users\Pochkarenok\anaconda3\envs\Text\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


INFO: Pandarallel will run on 12 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


In [3]:
import mmap
import re

def get_num_lines(file_path):
    fp = open(file_path, "r+")
    buf = mmap.mmap(fp.fileno(), 0)
    lines = 0
    while buf.readline():
        lines += 1
    return lines

In [4]:
N = get_num_lines("C:/Users/Pochkarenok/Text NLP/Otvety.txt")

In [5]:
N

7550926

In [6]:
# Преобразование файла вопросов-ответов в строчный вид

file_path_From = 'Otvety.txt'
file_path_To = 'prepared_answers.txt'


if not os.path.isfile(file_path_To):
    
    question = None
    written = False 

    with open(file_path_To, mode = 'w', encoding="utf-8") as fout:  
        with open(file_path_From, mode ="r",encoding="utf-8" ) as fin:
            for line in tqdm(fin):
                if line.startswith("---"):
                    written = False
                    continue
                if not written and question is not None:
                    fout.write(question.replace("\t", " ").strip() + " zzzzz " + line.replace("\t", " "))
                    written = True
                    question = None
                    continue
                if not written:
                    question = line.strip()




In [7]:
import re
# Препроцессинг текста
def preprocess_txt(line):
    line = re.sub("<[^>]+?>", '', line)
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return spls

In [8]:
# Обработка текста

sentences = []
morpher = MorphAnalyzer()
sw = set(get_stop_words("ru"))
exclude = set(string.punctuation)
counter = 0

file_path_from = 'prepared_answers.txt'
file_path_to = 'Otvety2.txt'

if not os.path.isfile(file_path_to):

    N = get_num_lines(file_path_from)
    with open(file_path_to, mode = 'w', encoding="utf-8") as fileto:
        with open(file_path_from, mode ="r",encoding="utf-8") as filefrom:
            for k in tqdm(range(N)):
                line = filefrom.readline()
                if line == '': break
                spls = preprocess_txt(line)
                sentences.append(spls)
                counter += 1
                if counter > 50_000:
                    break
                fileto.write(' '.join(spls)+'\n')
#

In [9]:
# Загрузить результат

sentences = []

file_path_from = 'Otvety2.txt'
if os.path.isfile(file_path_from):  
    N = get_num_lines(file_path_from) 
    with open(file_path_to, mode = 'r', encoding="utf-8") as filefrom:
        for line in tqdm(range(N)):
            line = filefrom.readline()
            if line == '': break
            sentences.append(line.split())


  0%|          | 0/50000 [00:00<?, ?it/s]

In [10]:
sentences[10:20]

[['пьяный',
  'везти',
  'большой',
  'трезвый',
  'zzzzz',
  'бог',
  'русь',
  'беречь',
  'пьяный',
  'дурак'],
 ['остановить',
  'выпадение',
  'волос',
  'беременность',
  'zzzzz',
  'столкнуться',
  'выпадение',
  'волос',
  'ждать',
  'ребёнок',
  'анализ',
  'показывать',
  'начинаться',
  'анемия',
  'скорее',
  'причина',
  'пропил',
  'курс',
  'железосодержащий',
  'препарат',
  'витамин',
  'параллельно',
  'применять',
  'маска',
  'dove',
  'борьба',
  'выпадение',
  'волос',
  'репейный',
  'масло',
  'месяц',
  'волос',
  'перестать',
  'выпадать',
  'лететь',
  'прямо',
  'клок',
  'посоветовать',
  'сменить',
  'расчёска',
  'деликатный'],
 ['программа',
  'измерение',
  'температура',
  'процессор',
  'компьютер',
  'знать',
  'существовать',
  'прога',
  'незнать',
  'достать',
  'помочь',
  'zzzzz',
  'everest'],
 ['существенный',
  'отличие',
  'ipad2',
  'ipad3',
  '3g',
  'память',
  'zzzzz',
  'слух',
  'новый',
  'дисплей',
  'рутина',
  'сумасшедший',
  'раз

In [11]:
vec = []
_ = [vec.extend(x)  for x in sentences[:100]]
vec = list(set(vec))
vec.sort()
vec[200:300]

['words',
 'wwwforumhouseruthreads143310',
 'x',
 'young',
 'your',
 'zzzzz',
 '«b',
 '«beautiful',
 '«bus',
 '«goodnight»',
 '«hey',
 '«let’s',
 '«russian',
 '«sam’s',
 '«special',
 '«triumph',
 '«we',
 '«welcome»',
 '«больной',
 '«быть',
 '«дажа',
 '«за',
 '«заглавие',
 '«звёздный»',
 '«иначе',
 '«крыжовник',
 '«крыжовник»',
 '«любовь',
 '«маленькая',
 '«о',
 '«он',
 '«прежде',
 '«самый',
 '«сандомирский',
 '«страна',
 '«человек',
 '«чешский',
 '«чтоба',
 '»',
 'авансовый',
 'авто',
 'автоматически',
 'автомобиль',
 'автоэлектрик',
 'администратор',
 'администрация',
 'администрировать',
 'академия',
 'аккуратный',
 'акронис',
 'активный',
 'актуальный',
 'актёр',
 'алгоритм',
 'аллизантин',
 'аллисата',
 'аллицин',
 'альбом',
 'американский',
 'аммиак',
 'аммомия',
 'анализ',
 'англ',
 'английский',
 'андрей',
 'анемия',
 'ането',
 'антибактериальный',
 'антивирусный',
 'аптека',
 'аренда',
 'аринушкемень',
 'армия',
 'армянский',
 'ассоциативный',
 'ассоциация',
 'атака',
 'аудитор

In [12]:
sentences[:20]

[['zzzzz', 'вопрос', 'тдв', 'отдыхать', 'лично', 'советовать', 'завести'],
 ['парень',
  'относиться',
  'цветной',
  'линза',
  'девушка',
  'зелёный',
  'глаз',
  'голубой',
  'zzzzz',
  'вобщий',
  'прикалывать',
  'тема'],
 ['делать',
  'найти',
  '2',
  'миллион',
  'рубль',
  'zzzzz',
  'счастие',
  'свалиться',
  'хороший',
  'пойти',
  'милиция',
  'заявить',
  'находка',
  'деньга',
  'тероть',
  'самый',
  'интересный',
  'неприменный',
  'искать',
  'поверьте',
  'найти',
  'видеть',
  'подобный',
  'нарваться',
  'бабушка',
  'помочий',
  'внук',
  'покупка',
  'квартира',
  'бандит',
  'разговаривать',
  'иначе',
  'бабушка',
  'милиция',
  'выбор',
  'шанс',
  'подарок',
  'выше',
  'котрый',
  'никто',
  'спросить',
  'хороший',
  'отдать',
  'хотяб',
  '500',
  'благотворительность',
  'дабы',
  'спугнуть',
  'удача'],
 ['эбу',
  'двенашка',
  'называться',
  'итэлма',
  'эбу',
  'zzzzz',
  'эбу',
  '—',
  'электронный',
  'блок',
  'управление',
  'двигатель',
  'автом

In [14]:
sentences = sentences[:50_000]

In [15]:
# Обучим модель FastText - но только на датасете Болталки (добавить продуктовый датасет)

file_path_from = 'ft_model'
if not os.path.isfile(file_path_from):  
    
    sentences = [i for i in tqdm(sentences) if len(i) > 2]
    modelFT = FastText(sentences=sentences, size=100, min_count=1, window=5)
    modelFT.save("ft_model")


In [16]:
# Загрузить модель

modelFT = FastText.load("ft_model")
ft_index = annoy.AnnoyIndex(100 ,'angular')

_Размерность вектора модели_. Если установить 100 — каждое слово в корпусе будет представлено в виде 100-мерного вектора, и т.д.

_Наименьшее допустимое количество символов в слове_, для которого будет создаваться векторное представление; так можно убрать частотные, но не очень значимые слова типа союзов и предлогов.

_Размер окна_. Этот параметр задаёт, сколько соседних слов считается частью контекста. Если выставить 50, то алгоритм возьмёт 50 слов спереди от слова и 50 слов сзади от слова.

In [16]:
list(set(get_stop_words("ru")))[:20]

['занято',
 'него',
 'суть',
 'вся',
 'наша',
 'нельзя',
 'тоже',
 'теми',
 'были',
 'года',
 'то',
 'быть',
 'вниз',
 'однажды',
 'никуда',
 'второй',
 'чем',
 'четвертый',
 'ты',
 'кому']

In [22]:
R = get_num_lines("Otvety2.txt")

In [23]:
R

50000

In [38]:
# Создаем Индексы для вопросов-ответов

file_path_from = 'speaker.ann'
file_path_to = 'Otvety2.txt'
#if not os.path.isfile(file_path_from):  
morpher = MorphAnalyzer()
sw = set(get_stop_words("ru"))
exclude = set(string.punctuation)
modelFT = FastText.load("ft_model")
ft_index = annoy.AnnoyIndex(100 ,'angular')
R = get_num_lines("Otvety2.txt")

index_map = {}
counter = 0
               
                
with open(file_path_to, mode = "r", encoding="utf-8") as f:
    for line in tqdm(range(R)):
        n_ft = 0
        line = f.readline()
        spls = line.split(" zzzzz ")
        index_map[counter] = spls[-1]
        question = preprocess_txt(spls[0])
        vector_ft = np.zeros(100)
        for word in question:
            if word in modelFT.wv:
                vector_ft += modelFT.wv[word]
                n_ft += 1
        if n_ft > 0:
            vector_ft = vector_ft / n_ft
        ft_index.add_item(counter, vector_ft)
        counter += 1

        if counter > 50_000:
            break

ft_index.build(10)
ft_index.save('speaker.ann')

with open("index_speaker.pkl", "wb") as f:
    pickle.dump(index_map, f)


  0%|          | 0/50000 [00:00<?, ?it/s]

In [39]:
#  Загрузим индексы
ft_index = annoy.AnnoyIndex(100, 'angular')
ft_index.load('speaker.ann')
index_map = pd.read_pickle("index_speaker.pkl")

In [40]:
np.random.permutation(100)

array([90, 15, 83, 43, 13, 72, 11, 29, 30, 70,  4, 94, 81, 75, 68, 99, 25,
        0,  5, 53, 46, 73, 67, 28, 66, 31,  8, 62, 86, 35, 63, 34,  1, 41,
       48, 33, 54, 26,  6, 20, 56, 93, 74, 45, 69, 88, 55, 80, 14, 97, 38,
       89, 71,  9, 59, 17, 61, 16, 49, 18, 84, 32, 78, 57, 52,  2, 50, 27,
       87, 79, 19, 96, 47, 91, 23, 64, 95, 39, 65, 10, 82, 58, 76, 40, 21,
        7, 92, 51, 44, 37, 77, 12, 60,  3, 42, 98, 22, 24, 85, 36])

In [41]:
#  Пример получения индексов
a = ft_index.get_nns_by_vector(np.random.permutation(100), 5, include_distances=True)
a

([1868, 11375, 1522, 15573, 34472],
 [1.3193811178207397,
  1.319551706314087,
  1.324871301651001,
  1.32960844039917,
  1.3316537141799927])

In [42]:
[index_map[x] for x in a[0]]

['httptfileruforumviewtopicphpt170932 искать фильм являться совместный проект студия disney успешный сериал bbc планета земля httprutrackerorgforumviewtopicphpt1878987\n',
 'феодосия\n',
 'товарищ должный прога itunes 702 скачать wwwappleru далее найти фильм формат mpeg4 качать конвертор dvdclone получать нужный формат далее музыка переносить айтюнза вкладка дополнительно выбирать конвертировать ipod далее плеер пытаться прямой качнуть dvd ipod выйти вопрос писать работа\n',
 'любитель панкрок музыка равнодушный панкрок музыка остальной напросто настратить in my humble opinion\n',
 'вообще сериалhttpwwwmnogodvdruproductinfophpcpath7173productsid21151absent1\n']

In [111]:
# https://gbcdn.mrgcdn.ru/uploads/asset/5209459/attachment/1b2f5aa57ff77e7c2d2ee26ceb09eb9e.csv

In [37]:
# Создадим модель продуктовых данных

shop_data = pd.read_csv("C:/Users/Pochkarenok/Text NLP/ProductsDataset.csv")
#shop_data = shop_data.iloc[:5000, :]

shop_data['text'] = shop_data['title'] + " " + shop_data["descrirption"]
shop_data['text'] = shop_data['text'].progress_apply(lambda x: preprocess_txt(str(x)))
shop_data.head()

  0%|          | 0/35548 [00:00<?, ?it/s]

,title,descrirption,product_id,category_id,subcategory_id,properties,image_links,text
0,Юбка детская ORBY,"Новая, не носили ни разу. В реале красивей чем...",58e3cfe6132ca50e053f5f82,22.0,2211,"{'detskie_razmer_rost': '81-86 (1,5 года)'}",http://cache3.youla.io/files/images/360_360/58...,"[юбка, детский, orby, новый, носить, реал, кра..."
1,Ботильоны,"Новые,привезены из Чехии ,указан размер 40,но ...",5667531b2b7f8d127d838c34,9.0,902,"{'zhenskaya_odezhda_tzvet': 'Зеленый', 'visota...",http://cache3.youla.io/files/images/360_360/5b...,"[ботильон, новыепривезти, чехия, указать, разм..."
2,Брюки,Размер 40-42. Брюки почти новые - не знаю как ...,59534826aaab284cba337e06,9.0,906,{'zhenskaya_odezhda_dzhinsy_bryuki_tip': 'Брюк...,http://cache3.youla.io/files/images/360_360/59...,"[брюки, размер, 4042, брюки, новый, знать, мер..."
3,Продам детские шапки,"Продам шапки,кажда 200р.Розовая и белая проданны.",57de544096ad842e26de8027,22.0,2217,"{'detskie_pol': 'Девочкам', 'detskaya_odezhda_...",http://cache3.youla.io/files/images/360_360/57...,"[продать, детский, шапка, продать, шапкикажда,..."
4,Блузка,"Темно-синяя, 42 размер,состояние отличное,как ...",5ad4d2626c86cb168d212022,9.0,907,"{'zhenskaya_odezhda_tzvet': 'Синий', 'zhenskay...",http://cache3.youla.io/files/images/360_360/5a...,"[блузка, темносиний, 42, размерсостояние, отли..."


In [43]:
# Подготовка для создания модели для определения домена данных

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression

vectorizer = CountVectorizer(ngram_range=(1, 2))

In [44]:
idxs = set(np.random.randint(0, len(index_map), len(shop_data)))
  # Вопрос-ответный домен
negative_texts = [" ".join(preprocess_txt(index_map[i])) for i in tqdm(idxs)]
   # Продуктовый домен
positive_texts = [" ".join(val) for val in tqdm(shop_data['text'].values)]

  0%|          | 0/25367 [00:00<?, ?it/s]

  0%|          | 0/35548 [00:00<?, ?it/s]

In [45]:
# ВО = 0, Прод = 1

dataset = negative_texts + positive_texts
labels = np.zeros(len(dataset))
labels[len(negative_texts):] = np.ones(len(positive_texts))

In [46]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(dataset, labels, test_size=0.2, 
                                                    stratify=labels, random_state=13)

In [47]:
# Модель

vectorizer.fit(dataset)

x_train_vec = vectorizer.transform(X_train)
x_test_vec = vectorizer.transform(X_test)

lr = LogisticRegression().fit(x_train_vec, y_train)

In [48]:
# Качество

from sklearn.metrics import accuracy_score
accuracy_score(y_true=y_test, y_pred=lr.predict(x_test_vec))

0.9770992366412213

In [49]:
# Добавим IDF взвешивание (для каждого слова найдем IDF вес)

In [50]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer().fit(X_train)

In [51]:
np.mean(tfidf_vect.idf_)

10.733433450767336

In [52]:
idfs = {v[0]: v[1] for v in zip(tfidf_vect.vocabulary_, tfidf_vect.idf_)}

In [53]:
len(idfs.keys())

130458

In [54]:
list(idfs.keys())[:5]

['кофта', 'zara', 'практический', 'новый', '100']

In [55]:
list(idfs.values())[:5]

[9.086061496558823,
 7.014988204549503,
 10.695499408992923,
 11.100964517101087,
 11.100964517101087]

In [56]:
# Создаем Индексы для продуктовых данных

file_path_from = 'shop.ann'
if not os.path.isfile(file_path_from):  
    
    
    ft_index_shop = annoy.AnnoyIndex(100 ,'angular')

    midf = np.mean(tfidf_vect.idf_)

    index_map_shop = {}
    counter = 0

    for i in tqdm(range(len(shop_data))):
        n_ft = 0
        index_map_shop[counter] = (shop_data.loc[i, "title"], shop_data.loc[i, "image_links"])
        vector_ft = np.zeros(100)
        for word in shop_data.loc[i, "text"]:
            if word in modelFT.wv:
                vector_ft += modelFT.wv[word] * idfs.get(word, midf)
                n_ft += idfs.get(word, midf)
        if n_ft > 0:
            vector_ft = vector_ft / n_ft
        ft_index_shop.add_item(counter, vector_ft)
        counter += 1

    ft_index_shop.build(10)
    ft_index_shop.save('shop.ann')

    file_path_from = 'index_shop.pkl'
    if not os.path.isfile(file_path_from):  
    
        with open("index_shop.pkl", "wb") as f:
            pickle.dump(index_map_shop, f)


In [57]:
# Загрузим индексы

midf = np.mean(tfidf_vect.idf_)

ft_index_shop = annoy.AnnoyIndex(100, 'angular')
ft_index_shop.load('shop.ann') 

index_map_shop = pd.read_pickle("index_shop.pkl")

In [58]:
# Основная функция преобразования текста в вектор х100

def embed_txt(txt, idfs, midf):
    n_ft = 0
    vector_ft = np.zeros(100)
    for word in txt:
        if word in modelFT.wv:
            vector_ft += modelFT.wv[word] * idfs.get(word, midf)
            n_ft += idfs.get(word, midf)
    return vector_ft / n_ft

In [59]:
# Пример получения индекса

ft_index_shop.get_nns_by_vector(np.ones(100)*20, 5, include_distances=True)

([25696, 35155, 20291, 19381, 11923],
 [1.3141052722930908,
  1.3173162937164307,
  1.322340726852417,
  1.3225866556167603,
  1.3229831457138062])

In [130]:
# Создание своего бота в телеграмм
# @botfather
# /start
# /newbot - create a new bot
# name1
# name1_BOT
#. ->. API

In [ ]:
stop

In [ ]:

updater = Updater("6887227931:AAEkLMeaZtSenLDwA9ApM6ec9QYd1msTYM8", use_context=True) # Мой токен API к Telegram
dispatcher = updater.dispatcher

def startCommand(update, context):
    context.bot.send_message(chat_id=update.message.chat_id, text='Привет, давай пообщаемся?')

def textMessage(update, context):
    
    input_txt = preprocess_txt(update.message.text)
    vect = vectorizer.transform([" ".join(input_txt)])
    prediction = lr.predict(vect)
    
    # ПРОД
    if prediction[0] == 1:
        vect_ft = embed_txt(input_txt, idfs, midf)
        ft_index_shop_val = ft_index_shop.get_nns_by_vector(vect_ft, 5)
        for item in ft_index_shop_val:
            title, image = index_map_shop[item]
            context.bot.send_message(chat_id=update.message.chat_id, text="title: {} image: {}".format(title, image))
        return
    
    # QA
    vect_ft = embed_txt(input_txt, {}, 1)
    ft_index_val, distances = ft_index.get_nns_by_vector(vect_ft, 1, include_distances=True)
    print(distances[0])
    
    # 
    if distances[0] > 10000.5:
        print(distances[0])
        context.bot.send_message(chat_id=update.message.chat_id, text="Не понимаю")
        return
    
    # Вопрос-Ответ
    context.bot.send_message(chat_id=update.message.chat_id, text=index_map[ft_index_val[0]])
        
start_command_handler = CommandHandler('start', startCommand)
text_message_handler = MessageHandler(Filters.text, textMessage)
dispatcher.add_handler(start_command_handler)
dispatcher.add_handler(text_message_handler)
updater.start_polling(clean=True)
updater.idle()

0.3139152228832245
0.3445718586444855
0.0001814075803849846
0.2781445384025574
0.34788623452186584
